In [21]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.33.2 --progress-bar off
!pip install -qqq langchain==0.0.299 --progress-bar off
!pip install -qqq xformers==0.0.21 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq tokenizers==0.11.1 --progress-bar off


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-pubsublite 1.8.3 requires overrides<7.0.0,>=6.0.1, but you have overrides 7.4.0 which is incompatible.
jupyterlab 4.0.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.1 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.1 requires jupyterlab<5.0.0a0,>=4.0.6, but you have jupyterlab 4.0.5 which is incompatible.
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


In [23]:
import pandas as pd
import json


In [24]:
from huggingface_hub import login
login()

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [25]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"
#MODEL_NAME = "meta-llama/Llama-2-7b-chat"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0.001})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [30]:
from langchain import PromptTemplate

template = """
<s>[INST] <<SYS>>
For a given user post sentence, does it show signs of the symptom. Answer in binary "yes" or 
"no", for every symptom. The symptoms are as follows:
[Little interest or pleasure in doing things,

Feeling down, depressed, or hopeless, 

Trouble falling or staying asleep, or sleeping too much, 
Feeling tired or having little energy, 

Poor appetite or overeating, 

Feeling bad about yourself or that you are a failure or
have let yourself or your family down, 

Trouble concentrating on things, such as reading the
newspaper or watching television,

Moving or speaking so slowly that other people could
have noticed. Or the opposite being so figety or
restless that you have been moving around a lot more
than usual,

Thoughts that you would be better off dead, or of
hurting yourself] .

Give the output as a python list of lists, with the list containing the symptom and the
answer, separated by a comma.
{text}:
<</SYS>>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [31]:
text = """
Even during my depression I’ll get these small mini bursts of optimism and motivation thinking, “maybe I can make it?! Maybe I can improve my life and be happy!” And I’ll go to the gym for one day, or run, or try to learn something new, but that’s short lasted. The second I complete the task all I can think is. “What’s the point? I’m probably going to die anyway, so why spend time improving myself?”

I honestly have no will to even live life, all my energy is spent just existing by distracting myself and spending money on stupid shit just to forget about life.

I feel stuck in purgatory. Not courageous enough to kill myself, not motivated enough to make myself happy. I’m stuck. Existing.
"""

In [33]:
print(prompt.format(text=text))

TypeError: format_prompt() missing 1 required positional argument: 'system_prompt'

In [ ]:
%%time
result = llm(prompt.format(text=text))

In [ ]:
print(result)

In [ ]:
# with open('/kaggle/input/primate/primate_dataset.json', 'r') as file:
#     data = json.load(file)

# Initialize lists for columns
post_titles = []
second_columns = []
i=0
# Process each entry in the JSON data

import pandas as pd
df = pd.read_csv('/kaggle/input/prompt-dataset/prompt_dataset_primate.csv')

for index, row in df.iterrows():
    # Extract post_title and post_text
    print(index)
    post_title = row['Title']
    post_text = row['Text']
    print(f'TITLE: {post_title}')
    #print(f'TEXT: {post_text}')

    # Process post_text using the get_ans function
    second_column_value = llm(prompt.format(text=post_text))

    # Append values to lists
    post_titles.append(post_title)
    second_columns.append(second_column_value)

# Create a DataFrame
df = pd.DataFrame({'Title': post_titles, 'Output': second_columns})

# Write DataFrame to CSV
#df.to_csv('abc.csv', index=False)


In [ ]:
# df.to_csv('llama2_zeroshot.csv', index=False)
# print('SUCCESS')

In [ ]:
# from langchain.prompts.few_shot import FewShotPromptTemplate
# from langchain.prompts.prompt import PromptTemplate

# examples = [
#   {
#     "question": "Who lived longer, Muhammad Ali or Alan Turing?",
#     "answer":
# """
# Are follow up questions needed here: Yes.
# Follow up: How old was Muhammad Ali when he died?
# Intermediate answer: Muhammad Ali was 74 years old when he died.
# Follow up: How old was Alan Turing when he died?
# Intermediate answer: Alan Turing was 41 years old when he died.
# So the final answer is: Muhammad Ali
# """
#   },
#   {
#     "question": "When was the founder of craigslist born?",
#     "answer":
# """
# Are follow up questions needed here: Yes.
# Follow up: Who was the founder of craigslist?
# Intermediate answer: Craigslist was founded by Craig Newmark.
# Follow up: When was Craig Newmark born?
# Intermediate answer: Craig Newmark was born on December 6, 1952.
# So the final answer is: December 6, 1952
# """
#   },
#   {
#     "question": "Who was the maternal grandfather of George Washington?",
#     "answer":
# """
# Are follow up questions needed here: Yes.
# Follow up: Who was the mother of George Washington?
# Intermediate answer: The mother of George Washington was Mary Ball Washington.
# Follow up: Who was the father of Mary Ball Washington?
# Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
# So the final answer is: Joseph Ball
# """
#   },
#   {
#     "question": "Are both the directors of Jaws and Casino Royale from the same country?",
#     "answer":
# """
# Are follow up questions needed here: Yes.
# Follow up: Who is the director of Jaws?
# Intermediate Answer: The director of Jaws is Steven Spielberg.
# Follow up: Where is Steven Spielberg from?
# Intermediate Answer: The United States.
# Follow up: Who is the director of Casino Royale?
# Intermediate Answer: The director of Casino Royale is Martin Campbell.
# Follow up: Where is Martin Campbell from?
# Intermediate Answer: New Zealand.
# So the final answer is: No
# """
#   }
# ]